In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import time
import numpy as np

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU.
    try:
        # Currently, memory growth needs to be the same across GPUs.
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

        # Use only the selcted gpu.
        tf.config.experimental.set_visible_devices(gpus[int(0)], 'GPU')
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized.
        print(e)

In [2]:
filter_shape = (206,206, 1, 300)
stride = 1

img_list_file = '../02 Deep Learning/02 Evaluation/list_of_img_in_OP_val_set_14-04.csv'

file = open(img_list_file)
file_paths = list(file)

file_paths = file_paths[0:1]

batch = np.empty([0,720,1280,1])

for path in file_paths:
    image_name = path.strip('\n')
    image = cv2.imread('../03 Data/Dataset2_onPallet/'+image_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = np.expand_dims(image, axis=(0, 3))
    batch = np.concatenate((image,batch),axis=0)
print(batch.shape)
batch = batch/255

(1, 720, 1280, 1)


In [ ]:
x = tf.constant(batch, dtype=tf.float32)
kernel_in = np.random.rand(206,206, 1, 300)
kernel = tf.constant(kernel_in, dtype=tf.float32)

total_time = 0
with tf.device('/GPU:0'):
    start = time.perf_counter()
    tf.nn.conv2d(x, kernel, 1, padding='VALID')
    end = time.perf_counter()
    total_time = end-start
print(total_time)

In [3]:
model = tf.keras.Sequential([layers.Conv2D(512,256, use_bias=False)])

In [4]:
start = time.perf_counter()
with tf.device('/GPU:0'):
    result = model.predict(batch)
end = time.perf_counter()
total_time = end-start
print(total_time)
print(result.shape)

21.010955298000226
(1, 465, 1025, 512)


In [ ]:
model(batch, training=False)